In [35]:
from datasets import load_dataset
DATASET_NAME='qasc'
dataset = load_dataset("allenai/qasc", "", split=["train", "validation"])

In [36]:
choice_len=max([len(x['label']) for x in dataset[1]['choices']])
choice_len

8

In [46]:
NUM_PERMUTE_QUESTION=10
EACH_HAS=50

In [47]:
# all_ques1=[x for x in dataset[0]['id']]
# all_ques2=[x for x in dataset[1]['id']]

In [48]:
# [x for x in all_ques1 if x in all_ques2]

In [49]:
def get_prompt(dataset, index,perm=None,  has_choice=False):
    try:
        json_line = dataset[index]
        question = json_line["question"]
        choices = json_line["choices"]
        choice_texts = choices["text"]
        choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
        candidates = " ".join(
            [
                f"({label if has_choice else ' '}) {text}"
                for text, label in zip(choice_texts, choices["label"])
            ]
        ).replace("\n", " ")
        answer_key = json_line["answerKey"]
        answer_key_idx = ord(answer_key[0]) - ord("A")
        answer_text = choice_texts[answer_key_idx]
        fact = f"{json_line['combinedfact']}{'' if json_line['combinedfact'][-1]=='.' else '.'}"
        prompt = f"{fact} {question} \\n {candidates}"
        return prompt, answer_text
    except:
        print('QUESTION: ',question,'ANSWER KEY', answer_key)

In [50]:
import itertools


all_permutes=list(itertools.permutations(list(range(choice_len))))

In [51]:
import random
from tqdm import tqdm, trange
import pickle
if NUM_PERMUTE_QUESTION:
    container=[]
    for ques_index in trange(NUM_PERMUTE_QUESTION):
        selection=random.sample(all_permutes,EACH_HAS)
        for perm in tqdm(selection):
            container.append(
                    get_prompt(dataset[1],ques_index,perm,  has_choice=False)
                )
    pickle.dump(container, open(f"{DATASET_NAME}_test_permute.pkl", "wb"))
else:
    for index, dataname in enumerate(['train','test']):
        container=[]
        for ques_index, ques in enumerate(tqdm(dataset[index])):
            container.append(
                    get_prompt(dataset[index], ques_index, has_choice=False)
                )
        pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 10/10 [00:00<00:00, 71.25it/s]
